In [1]:
import numpy as np
# import linecache
from datetime import datetime,timedelta
import pandas as pd
import sys
import time
import gc

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/util_dir/')

from time_systems import mjds_to_ymdhms
from time_systems import time_gps_to_tdt
#
from quaternions  import quat_trans_SBFtoRSW_to_SBFtoECI
from quaternions  import load_attitude_spire
from quaternions  import call_slerp_SpireAtt
#
from attitude_binary  import write_EXAT_binary



## Specify inputs


In [3]:
startEpoch  = "2018-11-08 23:00:00"  
stopEpoch   = "2018-11-10 01:00:00" 
interval    = 10  # seconds


sat = 'Spire83_test'

    



path_attitude     = "/data/data_geodyn/inputs/spire/data_Spire/attitude/20180923_20181209_TaskOrder3Data/"
filename          = "leoAtt_2018-09-23T00-12-00Z.9999999.083.log"
file__AttitudeLog = path_attitude+filename

# Prepare Spire data


In [4]:
#### 1. Load the attitude data that corresponds to the entire timeperiod
SpireDF = load_attitude_spire(file__AttitudeLog,
                                  startEpoch,
                                  stopEpoch)


#### 2. Convert from GPS time to TDT time
SpireDF['tim (tdt)'] = [time_gps_to_tdt(tim, leap_sec=37) 
                                 for tim in SpireDF['tim (gps)'] 
                                        ]


#### 3. Perform a coordinate transformation from SBF-->RSW to SBF-->ECI(j2000)
q_SBFtoECI = [quat_trans_SBFtoRSW_to_SBFtoECI(SpireDF['pos (eci)'].iloc[i], 
                                              SpireDF['vel (eci)'].iloc[i], 
                                              SpireDF['q (sbf)'].iloc[i]) 
                                          for i,val in enumerate(SpireDF['tim (gps)'])]
# Fix the negatives such that no scalar component is negative
SpireDF['q (SBFtoECI)'] = [-1*x if x[3]<0
                                else x
                                for x in q_SBFtoECI]


#### 4. Interpolate Quaternions to linearly spaced time series
exatt_quats = call_slerp_SpireAtt(SpireDF, 
                                   startEpoch, 
                                   stopEpoch, 
                                   interval )


##### Free up some memory:
del SpireDF
del q_SBFtoECI
gc.collect()


0

# Write EXAT binary 

## Prep EXAT inputs


In [5]:
print(startEpoch)
print(stopEpoch)


#### Initialize the satellite specific settings 
####     for the external attitude file conditions
exat_param={}

if sat=='Spire83_test':
    exat_param[sat]={}
    exat_param[sat]['SATID']    = 1804607.0  #2018-046F
    exat_param[sat]['num_sat']  = 1.
    exat_param[sat]['version']  = 1.
    
    ### Time info
    exat_param[sat]['startEpoch']  = float(pd.to_datetime(
                                        startEpoch,format='%Y-%m-%d %H:%M:%S'
                                        ).strftime(format='%y%m%d%H%M%S'))
    exat_param[sat]['stopEpoch']   = float(pd.to_datetime(
                                        stopEpoch, format='%Y-%m-%d %H:%M:%S'
                                        ).strftime(format='%y%m%d%H%M%S'))
    exat_param[sat]['interval']    = float(interval) 
    exat_param[sat]['startFrac_S'] = float(pd.to_datetime(
                                        startEpoch, format='%Y-%m-%d %H:%M:%S').microsecond)
    exat_param[sat]['stopFrac_S']  = float(pd.to_datetime(
                                        stopEpoch, format='%Y-%m-%d %H:%M:%S').microsecond)
    
    ### Panel info:
        # QQQ is the total # of separate movable panels   
        #      + antenna quaternion sets for this satellite 
        #      (excludes SBF to J2000 quaternion set which is 
        #      mandatory for each satellite in the file).
        # PPP is the number of movable panels represented for 
        #      this satellite.
        # AAA is the number of moveable antenna represented for 
        #      this satellite.  One quaternion set may represent 
        #      the attitude for up to two movable panels and 
        #      one movable antenna. 
    QQQ                          = int(   0   )
    PPP                          = int(   0   )
    AAA                          = int(   0   )
    exat_param[sat]['qqqpppaaa'] = float(f'{QQQ:03}{PPP:03}{AAA:03}')
    exat_param[sat]['panel_num'] = float(  0  )

    
    
file_exat = 'EXAT01'

2018-11-08 23:00:00
2018-11-10 01:00:00


## Write 

In [6]:
write_EXAT_binary(file_exat, 
                  exat_param[sat], 
                  exatt_quats['q (SBFtoECI)'],
                  exatt_quats['tim (tdt)'])



Reached end of attitude data.  Closing the File


In [7]:
# for i, val in enumerate(exatt_quats['q (SBFtoECI)'][:100]):
#     print(exatt_quats['q (SBFtoECI)'][i,0] )
#     print(exatt_quats['q (SBFtoECI)'][i,1] )
#     print(exatt_quats['q (SBFtoECI)'][i,2] )
#     print(exatt_quats['q (SBFtoECI)'][i,3] )


In [8]:
# #### Specify the input conditions




# # startEpoch     = 181108190109.0       #(YYMMDDHHMMSS.00)
# # stopEpoch      = 181111050109.0       #(YYMMDDHHMMSS.00)
# # start_str = pd.to_datetime(startEpoch, format='%Y-%m-%d %H:%M:%S').strftime(format='%y%m%d%H%M%S')
# # stop__str  = pd.to_datetime(stopEpoch,  format='%Y-%m-%d %H:%M:%S').strftime(format='%y%m%d%H%M%S')

# # startFrac_S    = pd.to_datetime(startEpoch, format='%Y-%m-%d %H:%M:%S').microsecond  #(fractional seconds, 00.SS)
# # stopFrac_S     = pd.to_datetime(stopEpoch, format='%Y-%m-%d %H:%M:%S').microsecond  #(fractional seconds, 00.SS)
# # intervalStep_S = float(interval)        #SSSSS.SSSSS


#     # QQQ is the total # of separate movable panels + antenna quaternion  
#     #     sets for this satellite (excludes SBF to J2000 quaternion set  
#     #     which is mandatory for each satellite represented in the file). 
# QQQ = 1 

#     # PPP is the number of movable panels represented for this satellite. 
# PPP = 2
#     # AAA is the number of moveable antenna represented for this  satellite.
#     #     One quaternion set may represent the attitude for up to two  
#     #     movable panels and one movable antenna.                
# AAA = 0

# qqqpppaaa = float(f'{QQQ:03}{PPP:03}{AAA:03}')
# #      QQQPPPAAA '1002000.0'

# panel_number = 0.  # zero indicates the non-moveable body

# #-------------------------------------------------------------

# # startDT = pd.to_datetime(startEpoch, format='%y%m%d%H%M%S')
# # stopDT  = pd.to_datetime(stopEpoch, format='%y%m%d%H%M%S')
# # freq_str = str(int(intervalStep_S))+"S"
# # len_dates = np.shape(pd.date_range(start=startDT, end=stopDT, freq=freq_str))[0]
# # print("Anticipating",len_dates,"data points." )
# # print("    Start:  ", startDT)
# # print("    Stop :  ", stopDT)
# # print("    Cadence:", freq_str,"(second)")
# # date_range = pd.date_range(start=start_epoch_trunc, end=stop_epoch_trunc, freq=freq_str)


#   

# Helpful Bits

Plot interpolated Quats

In [9]:
# import plotly.graph_objects as go
# from plotly.offline import plot, iplot
# from plotly.subplots import make_subplots
# import plotly.express as px
# import plotly.io as pio   ### Allows you to save plotly figs


# fig  = make_subplots(
#     rows=4, cols=1,
#     subplot_titles=(['qx','qy','qz','qw']),
#     shared_xaxes=False)

# ################S#######################################
# font_dict=dict(family='Arial',size=14,color='black')
# #######################################################

# col_r  =  "#ff7f0e"  # 'tab:orange'
# col_b  =  "#1f77b4"  # 'tab:blue'


# fig.add_trace(go.Scattergl(x=SpireDF['tim (tdt)'].values,
#                            y=np.array(SpireDF['q (SBFtoECI)'].values.tolist())[:,0] ,
#                            mode='markers',
#                            marker = dict(size=4, color=col_b),
#                            opacity=1,
#                            showlegend=False), row=1, col=1)
# fig.add_trace(go.Scattergl(x=SpireDF['tim (tdt)'].values,
#                            y=np.array(SpireDF['q (SBFtoECI)'].values.tolist())[:,1] ,
#                            mode='markers',
#                            marker = dict(size=4, color=col_b),
#                            opacity=1,
#                            showlegend=False), row=2, col=1)
# fig.add_trace(go.Scattergl(x=SpireDF['tim (tdt)'].values,
#                            y=np.array(SpireDF['q (SBFtoECI)'].values.tolist())[:,2] ,
#                            mode='markers',
#                            marker = dict(size=4, color=col_b),
#                            opacity=1,
#                            showlegend=False), row=3, col=1)
# fig.add_trace(go.Scattergl(x=SpireDF['tim (tdt)'].values,
#                            y=np.array(SpireDF['q (SBFtoECI)'].values.tolist())[:,3] ,
#                            mode='markers',
#                            marker = dict(size=4, color=col_b),
#                            opacity=1,
#                            showlegend=False), row=4, col=1)

# #################################################

# fig.add_trace(go.Scattergl(x=extatt_quats['tim (tdt)'],
#                            y=extatt_quats['q (SBFtoECI)'][:,0] ,
#                            mode='markers',
#                            marker = dict(size=2, color=col_r),
#                            opacity=1,
#                            showlegend=False), row=1, col=1)
# fig.add_trace(go.Scattergl(x=extatt_quats['tim (tdt)'],
#                            y=extatt_quats['q (SBFtoECI)'][:,1] ,
#                            mode='markers',
#                            marker = dict(size=2, color=col_r),
#                            opacity=1,
#                            showlegend=False), row=2, col=1)
# fig.add_trace(go.Scattergl(x=extatt_quats['tim (tdt)'],
#                            y=extatt_quats['q (SBFtoECI)'][:,2] ,
#                            mode='markers',
#                            marker = dict(size=2, color=col_r),
#                            opacity=1,
#                            showlegend=False), row=3, col=1)
# fig.add_trace(go.Scattergl(x=extatt_quats['tim (tdt)'],
#                            y=extatt_quats['q (SBFtoECI)'][:,3] ,
#                            mode='markers',
#                            marker = dict(size=2, color=col_r),
#                            opacity=1,
#                            showlegend=False), row=4, col=1)

# fig.update_xaxes( range=[pd.to_datetime(extatt_quats['tim (tdt)'][0] )-pd.to_timedelta(1,'h'),
#                          pd.to_datetime(extatt_quats['tim (tdt)'][-1])+pd.to_timedelta(1,'h')],
#                 )

# fig.update_layout(
#                   autosize=False,    width=1000,    height=950,
#                   legend= {'itemsizing': 'trace'},
#                   font=font_dict,
#                  )
# fig.show(config=dict({
#             'displayModeBar': False,
#             'responsive': False,
#             'staticPlot': False,
#             'displaylogo': False,
#             'showTips': False,
#             }))


## Repeats in the attitude file

In [10]:
# tim 2018 11  8  0  8 51.4090000 0
# sca  0.0064003782  0.0045788550 -0.3360005392  0.9418289160
# pvi   110.8517422  5122.5155000  4545.7920000    -6.3928726    -2.6903757     3.1860359
# tim 2018 11  8  0  8 51.4090000 0
# sca  0.0064003782  0.0045788550 -0.3360005392  0.9418289160
# pvi   110.8517422  5122.5155000  4545.7920000    -6.3928726    -2.6903757     3.1860359



# tim 2018 11  8  3 12 27.4990000 0
# sca -0.0061611997  0.0124282731 -0.9911857216  0.1317514438
# pvi  1950.5770000  5598.9030000  3431.4302500    -6.0466343    -0.7203881     4.6038013
# tim 2018 11  8  3 12 27.4990000 0
# sca -0.0061611997  0.0124282731 -0.9911857216  0.1317514438
# pvi  1950.5770000  5598.9030000  3431.4302500    -6.0466343    -0.7203881     4.6038013
# tim 2018 11  8  3 12 27.4990000 0
# sca -0.0061612039  0.0124282743 -0.9911856951  0.1317516428
# pvi  1950.5770000  5598.9030000  3431.4302500    -6.0466343    -0.7203881     4.6038013


## Write to Binary

In [11]:
# #### Specify the input conditions

# SATID          = 1807001. #1804607.0     #2018-046F
# num_satellites = 1.
# step_interval  = 1.       #seconds
# version_number = 1.



# startEpoch     = 181108190109.0       #(YYMMDDHHMMSS.00)
# stopEpoch      = 181111050109.0       #(YYMMDDHHMMSS.00)
# startFrac_S    = 00.18  #(fractional seconds, 00.SS)
# stopFrac_S     = 00.18  #(fractional seconds, 00.SS)
# intervalStep_S = 1.          #SSSSS.SSSSS


#     # QQQ is the total # of separate movable panels + antenna quaternion  
#     #     sets for this satellite (excludes SBF to J2000 quaternion set  
#     #     which is mandatory for each satellite represented in the file). 
# QQQ = 1 

#     # PPP is the number of movable panels represented for this satellite. 
# PPP = 2
#     # AAA is the number of moveable antenna represented for this  satellite.
#     #     One quaternion set may represent the attitude for up to two  
#     #     movable panels and one movable antenna.                
# AAA = 0

# qqqpppaaa = float(f'{QQQ:03}{PPP:03}{AAA:03}')
# #      QQQPPPAAA '1002000.0'

# panel_number = 0.  # zero indicates the non-moveable body

# #-------------------------------------------------------------

# startDT = pd.to_datetime(startEpoch, format='%y%m%d%H%M%S')
# stopDT  = pd.to_datetime(stopEpoch, format='%y%m%d%H%M%S')
# freq_str = str(int(intervalStep_S))+"S"
# len_dates = np.shape(pd.date_range(start=startDT, end=stopDT, freq=freq_str))[0]
# print("Anticipating",len_dates,"data points." )
# print("    Start:  ", startDT)
# print("    Stop :  ", stopDT)
# print("    Cadence:", freq_str,"(second)")
# # date_range = pd.date_range(start=start_epoch_trunc, end=stop_epoch_trunc, freq=freq_str)


In [12]:
# Q1 = Icesat2_att_body['q1'].values
# Q2 = Icesat2_att_body['q2'].values
# Q3 = Icesat2_att_body['q3'].values
# Q4 = Icesat2_att_body['q4'].values

In [13]:
# from scipy.io import FortranFile

# f = FortranFile('test_file', 'w')

# ### --------------------------------------------------------------------
# ###                   1) GENERAL HEADER RECORD  
# ### --------------------------------------------------------------------
# ###                   Read the first record, this is the header buffer
# ###                   Use 64-bit float datatype. 
# ###                   Each record contains 9, 64-bit words.
# ### --------------------------------------------------------------------

# record1_HeaderGeneral = [
#             -6666666.0,  #     1--- Record Indicator #(-6666666.00)
#    version_number  ,     #     2--- Version Number
#    num_satellites  ,     #     3--- Number of Satellites
#               0.0  ,     #     Not  used at  present  time
#               0.0  ,     #     Not  used at  present  time
#               0.0  ,     #     Not  used at  present  time
#               0.0  ,     #     Not  used at  present  time
#               0.0  ,     #     Not  used at  present  time
#               0.0  ,     #     Not  used at  present  time
#                 ]

# ### ---------------------------------------------------------------------
# ###                   2) SATELLITE INFORMATION RECORD                
# ### ---------------------------------------------------------------------
# ###                   The number of these records equals the number of 
# ###                   satellites from the General Header Record.  
# ###                   All of these records must directly follow the 
# ###                   above General Header Record.
# ### ---------------------------------------------------------------------

# record2_HeaderSatInfo = [
#         -7777777.0 ,     # 1--- Record Indicator       (-7777777.00)
#               0.0  ,     #      Not used at present time
#             SATID  ,     # 3--- Satellite ID
#    intervalStep_S  ,     # 4--- Interval              (SSSSS.SSSSS)
#        startEpoch  ,     # 5--- Start time            (YYMMDDHHMMSS.00)
#       startFrac_S  ,     # 6--- Start frac secs       (00.SS)
#         stopEpoch  ,     # 7--- Stop time             (YYMMDDHHMMSS.00) 
#        stopFrac_S  ,     # 8--- Stop frac secs        (00.SS)
#         qqqpppaaa  ,     # 9--- QQQPPPAAA     # QQQ is the total # of separate movable panels + antenna quaternion  
#                                               #      sets for this satellite (excludes SBF to J2000 quaternion set  
#                                               #      which is mandatory for each satellite represented in the file). 
#                                               # PPP is the number of movable panels represented for this satellite. 
#                                               # AAA is the number of moveable antenna represented for this  satellite.
#                                               #      One quaternion set may represent the attitude for up to two  
#                                               #      movable panels and one movable antenna.                
#                 ]

        

# ### ---------------------------------------------------------------------
# ###                   3) QUATERNION SET HEADER RECORD  
# ### ---------------------------------------------------------------------
# ###                   This header record must precede the quaternion 
# ###                   data records for a a particular set of quaternions.
# ### ---------------------------------------------------------------------

# record3_HeaderQuatInfo = [
#        -8888888.0  ,     # 1 --- Record Indicator      (-8888888.00)
#             SATID  ,     # 2 --- Satellite ID
#      panel_number  ,     # 3 --- Panel # (MMMNNN)      (0 if not moveable; MMM is 1st panel NNN is 2nd panel number)
#                0.  ,     # 4 --- AntennaLink number
#        startEpoch  ,     # 5 --- Start time            (YYMMDDHHMMSS.00)
#       startFrac_S  ,     # 6 --- Start frac secs       (00.SS)
#         stopEpoch  ,     # 7 --- Stop time             (YYMMDDHHMMSS.00) 
#        stopFrac_S  ,     # 8 --- Stop frac secs        (00.SS)
#    intervalStep_S  ,     # 9 --- Interval              (SSSSS.SSSSS) 
#                 ]


# f.write_record(np.array(record1_HeaderGeneral, dtype=float))
# f.write_record(np.array(record2_HeaderSatInfo, dtype=float))
# f.write_record(np.array(record3_HeaderQuatInfo, dtype=float))








# ### ---------------------------------------------------------------------
# ###                   4) DATA RECORD  
# ### ---------------------------------------------------------------------
# ###                   These records apply to a particular quaternion set
# ###                   and must follow (in time ascending order) the 
# ###                   quaternion set header record. 
# ### ---------------------------------------------------------------------
# ###                      Notes from Vol5 document:
# ###                      1) All quaternion sets(SBF to J2000 , or MVP to SBF , or MVA to SBF) 
# ###                         for a particular satellite ID must have the same interval, as well
# ###                         as the same start and stop times.
# ###                      2) Quaternion information must include 10 Integration steps 
# ###                         before & after the EPOCH of the satellite in the GEODYN run.
# ###                      3) q1 = -9999999.000 indicates quaternion information is missing
# ###                         for this time point. GEODYN will use the internal attitude module if selected.
# ###                      4) A panel or antenna must not have a designation number of 0. 
# ###                      5) GEODYN's quaternion definition is Eulerion. Therefore, 
# ###                         the scalar argument is defined as positive. Also, the quaternions
# ###                         should be provided without any discontinuities to provide for proper interpolation.


# for i, val in enumerate(Icesat2_att_body['Dates'].values):
#     record4_Data = [
#                      0.,     #       Not used at present time
#                      0.,     #       Not used at present time
#                  Q1[i] ,     # 3 --- Q1      [  sin (/2)n1  ]
#                  Q2[i] ,     # 4 --- Q2      [  sin (/2)n2  ]
#                  Q3[i] ,     # 5 --- Q3      [  sin (/2)n3  ]
#                  Q4[i] ,     # 6 --- Q4      [  cos (/2)    ]
#                      0.,     #       Not used at present time
#                      0.,     #       Not used at present time
#                      0.,     #       Not used at present time
#                     ]
#     f.write_record(np.array(record4_Data, dtype=float))


# print('Reached end of Data sets.  Closing the File')
# f.close()


In [14]:
# f = FortranFile('test_file', 'r')
# print(f.read_record(float))
# print(f.read_record(float))
# f.close()


## Dates

In [15]:
# datestart_string = str(header['SATINFO_01']['Start time (YYMMDDHHMMSS.00)']).split('.')[0]
# datestop_string  = str(header['SATINFO_01']['Stop time (YYMMDDHHMMSS.00)']).split('.')[0]
# date_start       = pd.to_datetime(datestart_string, format='%y%m%d%H%M%S')
# date_stop        = pd.to_datetime(datestop_string, format='%y%m%d%H%M%S')
# interval         = header['SATINFO_01']['Interval (SSSSS.SSSSS)']

# # date_start+pd.to_timedelta(interval,'s')

# Dates = []

# for i, val in enumerate(data[1]['q1']):
#     if i == 0:
#         Dates.append(date_start)
#     else:
#         Dates.append(Dates[-1]+pd.to_timedelta(interval,'s'))

    
    

